In [ ]:
# -----------------------------------------------------------------------------
# 📘 Notebook: 05_clustering_and_explainability.ipynb
#
# Purpose:
#   Discover patterns in runs via clustering and produce interpretable
#   summaries of each cluster's characteristics.
#
# Inputs :
#   - data/strava/processed/run_summary_features.parquet  (Stage 4 output)
#
# Outputs:
#   - data/strava/processed/run_clusters.parquet
#   - (optional) data/strava/processed/cluster_profiles.csv
#
# Steps:
#   1) Load feature dataset and define feature matrix
#   2) Quality checks: NaNs, scaling confirmation
#   3) Model selection: find a reasonable k via silhouette
#   4) Fit clustering (KMeans baseline)
#   5) 2D projection (PCA) for visualization
#   6) Cluster profiling (means, z-scores, rankings)
#   7) Lightweight explainability proxy (tiny decision tree)
#   8) Export artifacts (+ optional Neo4j write-back)
#   9) Brief summary
#
# Notes:
#   - Keep it simple and robust; add fancy variants (DBSCAN/HDBSCAN/UMAP) later.
#   - All plots are sanity checks; interpret with domain sense (training context).
# -----------------------------------------------------------------------------


In [ ]:
# --- 1) Load dataset & pick features ----------------------------------------
from pathlib import Path
import pandas as pd
import numpy as np

in_path = Path("../data/strava/processed/run_summary_features.parquet")
df = pd.read_parquet(in_path)
print(f"✅ Loaded {len(df):,} runs × {len(df.columns)} columns")

# Candidate feature set (numeric, scaled in Stage 4).
# If you changed names in Stage 4, update here to match.
feature_cols = [
    "avg_pace",
    "avg_cadence",
    "elevation_gain",
    "fatigue_index",
    "pace_variability",
    "elev_ratio",
]

# Keep a copy of IDs/dates for later merging/exports
meta_cols = ["run_id", "date", "total_distance_km", "duration_min", "weekday", "month"]
available_meta = [c for c in meta_cols if c in df.columns]


In [ ]:
# --- 2) Quality checks: missingness & scaling confirmation -------------------
# Interpretation tip:
# - Clustering is allergic to NaNs; either drop rows with NaNs in features
#   or impute. Here we impute safely to keep all runs.
from sklearn.impute import SimpleImputer

X_raw = df[feature_cols]
missing_counts = X_raw.isna().sum().sort_values(ascending=False)
print("NaNs per feature:\n", missing_counts.to_string())

imputer = SimpleImputer(strategy="mean")
X = imputer.fit_transform(X_raw)

# Optional quick scale check: If you DIDN'T scale in Stage 4, uncomment:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

print("✅ Feature matrix ready:", X.shape)


In [ ]:
# --- 3) Model selection: silhouette sweep for k ------------------------------
# Interpretation tip:
# - Silhouette score (0..1) measures how well-separated clusters are.
# - Look for an “elbow” or local maximum across k values.
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

k_values = list(range(2, 10))
scores = []
for k in k_values:
    km = KMeans(n_clusters=k, n_init="auto", random_state=42)
    labels = km.fit_predict(X)
    s = silhouette_score(X, labels)
    scores.append(s)

plt.figure(figsize=(6,4))
plt.plot(k_values, scores, marker="o")
plt.title("Silhouette score vs. k")
plt.xlabel("k (number of clusters)")
plt.ylabel("Silhouette score")
plt.grid(True)
plt.show()

best_k = k_values[int(np.argmax(scores))]
print(f"🧭 Suggested k (by silhouette): {best_k}  |  scores={['{:.3f}'.format(v) for v in scores]}")


Interpretation: The silhouette curve peaks strongly at k = 2, indicating two clearly distinct run patterns. Beyond that, cluster separation deteriorates sharply, so further partitioning would fragment coherent patterns rather than reveal new ones.

In [ ]:
# --- 4) Fit baseline KMeans clustering --------------------------------------
# Interpretation tip:
# - This assigns each run to a cluster. We'll profile clusters next to
#   understand their training “personality” (e.g., long hills vs recovery).
k = best_k  # you can override manually after seeing the curve
kmeans = KMeans(n_clusters=k, n_init="auto", random_state=42)
labels = kmeans.fit_predict(X)

df_clusters = df.copy()
df_clusters["cluster"] = labels
print(df_clusters["cluster"].value_counts().sort_index())


In [ ]:
# --- 5) 2D projection via PCA for visualization -----------------------------
# Interpretation tip:
# - This is just for visualization. Clusters that look separated in PCA space
#   are typically distinct in multivariate space, but PCA flattens reality.
from sklearn.decomposition import PCA

pca = PCA(n_components=2, random_state=42)
proj = pca.fit_transform(X)

plt.figure(figsize=(7,5))
scatter = plt.scatter(proj[:,0], proj[:,1], c=labels, s=35, alpha=0.8, cmap="tab10")
plt.title("PCA projection colored by cluster")
plt.xlabel("PC1"); plt.ylabel("PC2")
plt.colorbar(scatter, label="cluster")
plt.show()

print("Explained variance ratio:", pca.explained_variance_ratio_.round(3))


In [ ]:
# --- 6) Cluster profiling: means, z-scores, and feature ranking -------------
# Interpretation tip:
# - Means: absolute values per feature per cluster.
# - Z-scores: how far a cluster's mean sits from the global mean (std units).
# - Ranking: which features most distinguish each cluster.

# Compute cluster means (feature space)
cluster_means = (
    df_clusters.groupby("cluster")[feature_cols]
    .mean()
    .sort_index()
)

# Global mean/std for z-score normalization
global_mean = df[feature_cols].mean()
global_std  = df[feature_cols].std(ddof=0).replace(0, 1.0)

cluster_z = (cluster_means - global_mean) / global_std

# Rank features within each cluster by absolute z-score (distinctiveness)
ranked_features = (
    cluster_z.abs()
    .apply(lambda s: s.sort_values(ascending=False).index.tolist(), axis=1)
    .to_frame(name="feature_rank")
)

# Merge a compact profile table
profile = cluster_means.round(3)
profile["top_features"] = ranked_features["feature_rank"].apply(lambda xs: ", ".join(xs[:3]))

print("📊 Cluster means (selected):")
display(profile)

# Optional: also show z-scores
print("📈 Cluster z-scores (distinctiveness):")
display(cluster_z.round(2))


In [ ]:
# --- 7) Lightweight explainability proxy ------------------------------------
# Interpretation tip:
# - Train a tiny decision tree to predict cluster from features.
# - The resulting feature importances are a proxy for what separates clusters.
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(max_depth=3, random_state=42)
tree.fit(X, labels)
importances = pd.Series(tree.feature_importances_, index=feature_cols).sort_values(ascending=False)

ax = importances.plot(kind="barh", figsize=(6,4), title="Feature importance (Decision Tree)")
ax.invert_yaxis()
plt.show()

print("Top separating features:\n", importances.round(3).to_string())


In [ ]:
# --- 8) Export artifacts (+ optional Neo4j write-back) ----------------------
out_parquet = Path("../data/strava/processed/run_clusters.parquet")
out_parquet.parent.mkdir(parents=True, exist_ok=True)

# Minimal, analysis-friendly export
export_cols = available_meta + feature_cols + ["cluster"]
df_clusters[export_cols].to_parquet(out_parquet, index=False)
print(f"✅ Saved cluster assignments → {out_parquet.resolve()}")

# Optional: write cluster IDs into Neo4j (uncomment to use)
# from neo4j import GraphDatabase
# import os
# from dotenv import load_dotenv
# load_dotenv()
# NEO4J_URI  = os.getenv("NEO4J_URI")
# NEO4J_USER = os.getenv("NEO4J_USER")
# NEO4J_PASS = os.getenv("NEO4J_PASS")
# driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASS))
# def set_cluster(tx, rid, cid):
#     tx.run("MATCH (r:Run {run_id:$rid}) SET r.cluster=$cid", rid=rid, cid=int(cid))
# with driver.session() as s:
#     for rid, cid in df_clusters[["run_id","cluster"]].itertuples(index=False):
#         s.execute_write(set_cluster, rid, cid)
# driver.close()
# print("✅ (Optional) Wrote cluster labels to Neo4j")


In [ ]:
# --- 9) Brief summary --------------------------------------------------------
n_runs = len(df_clusters)
n_clusters = df_clusters["cluster"].nunique()
sizes = df_clusters["cluster"].value_counts().sort_index().to_dict()

print("──────────────────────────────────────────────────────────────────────────")
print("Stage 5 Summary")
print(f"- Runs clustered         : {n_runs}")
print(f"- Clusters discovered    : {n_clusters}")
print(f"- Cluster sizes          : {sizes}")
print("- Top separating features: ")
for f, v in pd.Series(tree.feature_importances_, index=feature_cols)\
            .sort_values(ascending=False).round(3).items():
    if v > 0:
        print(f"   • {f}: {v}")
print("──────────────────────────────────────────────────────────────────────────")

# Interpretation pointers:
# - Check 'Cluster means' and 'Cluster z-scores' to label clusters (e.g., 'Hilly long runs', 'Recovery spins').
# - If silhouette is flat or low, try a different k, different features, or DBSCAN.
# - For presentation, pair PCA plot with per-cluster histograms of pace/cadence.


In [ ]:
display(profile)


## 🏁 Stage 5 Summary — Clustering and Explainability Insights

**Data coverage**  
A total of **697 runs** were analyzed after preprocessing and feature engineering.  
Each run was assigned to one of **two clusters**, the number suggested by the silhouette analysis (peak score ≈ 0.66).

**Cluster composition**  
| Cluster | Runs | Approx. Share | Character |
|:--------:|-----:|:--------------|:-----------|
| 0 | 630 | ~90 % | Predominant, steady training pattern |
| 1 | 67  | ~10 % | Distinct minority of high-intensity or terrain-specific runs |

**Feature separation (Decision-Tree importance)**  
- **avg_cadence (0.71)** — the primary discriminator.  
  Higher cadence = faster, rhythmically tighter sessions; lower cadence = relaxed endurance runs.  
- **elev_ratio (0.29)** — secondary factor describing hilliness or elevation variation per kilometre.

**Interpretation of cluster behavior**  
- **Cluster 0** likely represents *steady endurance or easy runs* — moderate cadence, smoother pacing, relatively uniform elevation.  
- **Cluster 1** captures *performance or quality sessions* — higher cadence, greater variability, or steeper profiles (intervals, hills, or tempo work).

**Overall insight**  
Your training history naturally organizes into **two dominant performance regimes**:
1. A large body of consistent aerobic runs forming the structural base of your training.  
2. A smaller but distinctive set of faster or more intense efforts that drive adaptation.

These empirical patterns align with coaching intuition — the data now provides quantitative confirmation.  
Future stages (6 → modeling, 7 → dashboard/Neo4j integration) will build on this foundation to predict outcomes and visualize how intensity, cadence, and elevation interact over time.
